# 实战 MNIST 手写数字识别

**手动计算准确度**

In [4]:
import  tensorflow as tf
from    tensorflow import keras
from    tensorflow.keras import layers, optimizers, datasets

### 1. 全链路层
**准备数据**

In [5]:
(x, y), (x_val, y_val) = datasets.mnist.load_data()
x = tf.convert_to_tensor(x, dtype=tf.float32) / 255.
y = tf.convert_to_tensor(y, dtype=tf.int32)
y = tf.one_hot(y, depth=10)
print(x.shape, y.shape)

(60000, 28, 28) (60000, 10)


In [6]:
train_dataset = tf.data.Dataset.from_tensor_slices((x, y))
train_dataset = train_dataset.batch(200)

**建立模型(未编译)**
- 不能使用 evaluate predict 方法

In [7]:
model = keras.Sequential([
    layers.Dense(512, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(10)
])
# 优化器， 设置学习速率
optimizer = optimizers.SGD(learning_rate=0.001)

**训练模型**

In [8]:
def train_epoch(epoch):
    for step, (x, y) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            # [b, 28, 28] => [b, 784]
            x = tf.reshape(x, (-1, 28*28))
            # Step1. compute output
            # [b, 784] => [b, 10]
            out = model(x)
            # Step2. compute loss
            loss = tf.reduce_sum(tf.square(out - y)) / x.shape[0]
        
        # Step3. optimize and update w1, w2, w3, b1, b2, b3
        grads = tape.gradient(loss, model.trainable_variables)
        # w' = w - lr * grad
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        
        if step % 100 == 0:
            print(epoch, step, 'loss=', loss.numpy())

In [9]:
for epoch in range(5):
    train_epoch(epoch)

0 0 loss= 1.6406361
0 100 loss= 0.9647783
0 200 loss= 0.793602
1 0 loss= 0.67765856
1 100 loss= 0.6875607
1 200 loss= 0.60181135
2 0 loss= 0.55328506
2 100 loss= 0.59690267
2 200 loss= 0.518593
3 0 loss= 0.4930342
3 100 loss= 0.54453874
3 200 loss= 0.46801957
4 0 loss= 0.45505404
4 100 loss= 0.5079852
4 200 loss= 0.43304098


In [7]:
import numpy as np

**测试**

In [8]:
def predict(X):
    X = tf.reshape(X, (-1, 28*28))
    y_predict = model(X)
    return tf.argmax(y_predict, axis=1)

def accuracy(X_test, y_test):
    y_pred = predict(X_test)
    return np.sum(y_pred == y_test) / len(y_test)

In [9]:
accuracy(x_val, y_val)

0.8446